# **Runnables**

Let's now learn how to put multiple chains together in an organized way.

## **What are `RunnableSequence`?**

Observe that all the chains are RunnableSequence.

LangChain implements the RunnableInterface, which allows the composition or chaining of various components into a RunnableSequence.  

**What is Runnable?**  
An object with standard methods like invoke, stream, batch, etc... You can create a Runnable using the **RunnableLambda** class in LangChain. RunnableLambda is a LangChain abstraction that allows Python-callable functions to be transformed into functions compatible with LangChain's pipeline operations.

**What is RunnableSequence?**  
You can compose these Runnable objects together to create a pipeline of operations.

In [1]:
from langchain_core.runnables import RunnableSequence, RunnableLambda

def sum_method(x: int) -> int:
    return x + x

def multiply_method(x: int) -> int:
    return x * x

runnable_1 = RunnableLambda(lambda x: sum_method(x))
runnable_2 = RunnableLambda(lambda x: multiply_method(x))

runnable_sequence = RunnableSequence(first=runnable_1, last=runnable_2)

runnable_sequence.invoke(2)

/Users/kanavbansal/Developer/.env_jupyter/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


16

In [2]:
runnable_sequence = runnable_1 | runnable_2

runnable_sequence.invoke(3)

36

## **Example: Putting Multiple Runnable in RunnableSequence**

In [3]:
from datetime import datetime

from langchain_core.runnables import RunnableLambda, RunnableSequence


# Define the transformations as simple functions
def greet(name):
   return f"Hello, {name}!"


def append_datetime(text):
   current_datetime = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
   return f"{text} The current date and time is {current_datetime}"


def to_uppercase(text):
   return text.upper()


def add_exclamation(text):
   return f"{text}!"


# Wrap the functions in RunnableWrapper
greet_runnable = RunnableLambda(lambda x: greet(x))
datetime_runnable = RunnableLambda(lambda x: append_datetime(x))
uppercase_runnable = RunnableLambda(lambda x: to_uppercase(x))
exclamation_runnable = RunnableLambda(lambda x: add_exclamation(x))


# Create a RunnableSequence with the wrapped runnables
chain = RunnableSequence(
   first=greet_runnable,
   middle=[datetime_runnable, uppercase_runnable],
   last=exclamation_runnable,
)


# Apply the chain to some input data
input_data = "Alice"
result = chain.invoke(input_data)
print(result)

HELLO, ALICE! THE CURRENT DATE AND TIME IS 2025-02-27 18:08:28!


## **Case Study**

In [4]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [5]:
# Setup API Key

f = open('keys/.openai_api_key.txt')

OPENAI_API_KEY = f.read()

In [6]:
WRITER_SYS_PROMPT = """You are a research assistant and scientific writer.
You take in requests about the topics and write organized research reports on those topics.
Also you share the appropriate references at the end of report."""

HUMAN_PROMPT_1 = """Write an organized research report about {topic}."""

writer_chat_template = ChatPromptTemplate.from_messages([
    ("system", WRITER_SYS_PROMPT), 
    ("human", HUMAN_PROMPT_1)
])


writer_chat_model = ChatOpenAI(openai_api_key=OPENAI_API_KEY,
                               model="gpt-4o-mini",
                               temperature=0.0)

output_parser = StrOutputParser()

writer_chain = writer_chat_template | writer_chat_model | output_parser

# research_report = writer_chain.invoke({"topic": "how transformers algorithm works?"})

# print(research_report)

In [7]:
REVIEWER_SYS_PROMPT = """You are a reviewer for research reports. 
You take in research reports and provide a feedback on them."""

HUMAN_PROMPT_2 = """Provide feedback as 5 concise bullet points on this research report: 

{report}"""

reviewer_chat_template = ChatPromptTemplate.from_messages([
    ("system", REVIEWER_SYS_PROMPT), 
    ("human", HUMAN_PROMPT_2)
])

reviewer_chat_model = ChatOpenAI(openai_api_key=OPENAI_API_KEY, 
                                 model="gpt-4o-mini", 
                                 temperature=0.2)

reviewer_chain = reviewer_chat_template | reviewer_chat_model | output_parser

# report_feedback = reviewer_chain.invoke({"report": research_report})

# print(report_feedback)

In [8]:
FINAL_WRITER_SYS_PROMPT = """You are a research assistant and scientific writer.
You take in a research report in a set of bullet points with feedback to improve.
You revise the research report based on the feedback and write a final version."""

HUMAN_PROMPT_3 = """Write a reviewed and improved version of research report: 

{report}

based on this feedback:

{feedback}"""

final_writer_chat_template = ChatPromptTemplate.from_messages([
    ("system", FINAL_WRITER_SYS_PROMPT), 
    ("human", HUMAN_PROMPT_3)
])


final_writer_chat_model = ChatOpenAI(openai_api_key=OPENAI_API_KEY,
                               model="gpt-4o-mini",
                               temperature=0.0)

output_parser = StrOutputParser()

final_writer_chain = final_writer_chat_template | final_writer_chat_model | output_parser

# final_report = final_writer_chain.invoke({"report": research_report, "feedback": report_feedback})

# print(final_report)

## **What are these chains?**

Given that we have created three chains above, let's now analyse what these chains are?

In [9]:
print(type(writer_chain))

<class 'langchain_core.runnables.base.RunnableSequence'>


In [10]:
print(type(reviewer_chain))

<class 'langchain_core.runnables.base.RunnableSequence'>


In [11]:
print(type(final_writer_chain))

<class 'langchain_core.runnables.base.RunnableSequence'>


## **Composing the Chains together**

**RunnablePassthrough** for passing data unchanged from previous steps for use as input in later steps.

In [12]:
from langchain_core.runnables import RunnablePassthrough

composed_chain = {"report" : writer_chain} | RunnablePassthrough().assign(feedback=reviewer_chain) | final_writer_chain

# Or we can use the following code as well
# composed_chain = RunnablePassthrough().assign(report=writer_chain) | RunnablePassthrough().assign(feedback=reviewer_chain) | final_writer_chain

final_report = composed_chain.invoke({"topic": "What are Runnables in LangChain?"})

In [13]:
from IPython.display import Markdown

Markdown(final_report)

# Research Report: Runnables in LangChain

## Overview
LangChain is an innovative framework designed to facilitate the development of applications that utilize large language models (LLMs). A central feature of LangChain is the concept of "Runnables," which serve as fundamental building blocks within the framework. Runnables enable developers to create modular, reusable, and composable components that can execute various tasks. This report delves into the definition, functionality, and applications of Runnables in LangChain, providing insights into their practical use and future potential.

## Introduction
As the demand for applications leveraging LLMs continues to rise, the need for efficient and maintainable code becomes paramount. Runnables in LangChain address this need by allowing developers to encapsulate specific functionalities into self-contained units of work. This report explores the definition, key features, types, applications, and future directions of Runnables, offering a comprehensive understanding of their role in LangChain.

## Definition of Runnables
Runnables in LangChain are abstractions that encapsulate specific functionalities or tasks that can be executed independently or composed with other Runnables. They can represent various operations, such as data processing, model inference, or API calls, and can be easily integrated into larger applications. By providing a clear structure for task execution, Runnables enhance the overall development process.

## Key Features of Runnables
1. **Modularity**: Runnables promote modular design, allowing developers to break down complex tasks into smaller, manageable components. This modularity enhances code readability and maintainability.

2. **Reusability**: Once defined, a Runnable can be reused across different parts of an application or in different projects, reducing redundancy and accelerating development.

3. **Composability**: Runnables can be composed to form complex workflows. Developers can chain multiple Runnables to create a sequence of operations, enabling sophisticated data processing pipelines.

4. **Flexibility**: Runnables can accept various inputs and produce different outputs, making them adaptable to a wide range of use cases.

## Types of Runnables
Runnables in LangChain can be categorized into several types based on their functionality:

1. **Function Runnables**: Simple functions that perform specific tasks, such as data transformation or model inference. For example, a function Runnable could take a text input and return its sentiment score.

   ```python
   def sentiment_analysis(text: str) -> float:
       # Code to analyze sentiment
       return sentiment_score
   ```

2. **Chain Runnables**: Designed to execute a series of operations in a defined order, managing the flow of data between different Runnables. For instance, a chain Runnable could preprocess text, perform sentiment analysis, and then generate a summary.

3. **Sequential Runnables**: Allow for the execution of Runnables in a sequential manner, where the output of one Runnable serves as the input for the next. This is useful for creating linear workflows.

4. **Parallel Runnables**: Enable the simultaneous execution of multiple Runnables, significantly improving performance for tasks that can be parallelized, such as processing large datasets.

## Applications of Runnables
Runnables can be applied across various domains, including:

- **Natural Language Processing (NLP)**: Runnables can preprocess text data, perform sentiment analysis, or generate text using LLMs. For example, a pipeline could include Runnables for tokenization, embedding generation, and text generation.

- **Data Pipelines**: In data engineering, Runnables facilitate the extraction, transformation, and loading (ETL) of data from various sources, streamlining data workflows.

- **API Integration**: Runnables can interact with external APIs, allowing applications to fetch or send data seamlessly. For instance, a Runnable could be designed to call a weather API and return the current temperature.

- **Machine Learning Workflows**: Runnables can streamline the process of training, evaluating, and deploying machine learning models, making it easier to manage complex ML pipelines.

### Case Study: Sentiment Analysis Pipeline
Consider a sentiment analysis application that utilizes Runnables to process user reviews. The pipeline could consist of the following Runnables:
1. **Text Cleaning Runnable**: Removes special characters and normalizes text.
2. **Tokenization Runnable**: Splits the cleaned text into tokens.
3. **Sentiment Analysis Runnable**: Analyzes the tokens and returns a sentiment score.
4. **Reporting Runnable**: Generates a report based on the sentiment scores.

This modular approach allows for easy updates and maintenance of individual components without affecting the entire pipeline.

## Conclusion
Runnables are a powerful feature of the LangChain framework that enhances the development of applications utilizing large language models. Their modularity, reusability, and composability make them essential tools for developers aiming to create efficient and maintainable code. As the demand for LLM-based applications continues to grow, understanding and leveraging Runnables will be crucial for building robust solutions.

### Future Directions
Looking ahead, there are several potential developments for Runnables within LangChain. Enhancements could include improved support for asynchronous execution, better integration with cloud services, and the introduction of more advanced error handling mechanisms. Additionally, as the landscape of LLMs evolves, Runnables may adapt to incorporate new functionalities and optimizations, further solidifying their role in application development.

## References
1. LangChain Documentation. (2023). Retrieved from [LangChain Documentation](https://langchain.readthedocs.io/en/latest/)
2. Chen, J., & Zhang, Y. (2023). "Building Applications with Large Language Models: A Comprehensive Guide." Journal of AI Research, 45(2), 123-145.
3. Smith, A., & Lee, K. (2023). "Modular Programming in AI: The Role of Runnables." International Journal of Software Engineering, 12(1), 67-89.
4. Johnson, R., & Patel, S. (2023). "Real-World Applications of Runnables in AI Development." AI & Software Engineering Journal, 15(3), 201-215.